In [127]:
import pandas as pd
import numpy as np
import shutil
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.inception_v3 import InceptionV3

In [132]:
NUM_CLASSES = 7
SHORT_EPOCH = 5
LONG_EPOCH = 50
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
batch_size = 32

In [39]:
df = pd.read_csv("./labels/labels.csv")

In [47]:
df.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
data = df.as_matrix()

In [42]:

# x is your dataset
training_size = 7000
np.random.shuffle(data)
training, test = data[:training_size,:], data[training_size:,:]

In [43]:
training_vals = np.delete(training, 0,1)
test_vals = np.delete(test, 0,1)

In [52]:
training[0][0]

'ISIC_0030176'

In [45]:
training_vals[0].argmax(axis=0)

1

In [46]:
training[0]

array(['ISIC_0030176', 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], dtype=object)

In [72]:
def switch_demo(argument):
    switcher = {
        0: "MEL",
        1: "NV",
        2: "BCC",
        3: "AKIEC",
        4: "BKL",
        5: "DF",
        6: "VASC",
    }
    return (switcher.get(argument, "Invalid month"))

In [73]:
switch_demo(training_vals[0].argmax(axis=0))

'NV'

In [74]:
switch_demo(training_vals[0].argmax(axis=0))

'NV'

In [77]:
for i in range(len(training)):
    d_class = switch_demo(training_vals[i].argmax(axis=0))
    shutil.move("/home/john/Documents/skin/data/"+training[i][0]+".jpg", "/home/john/Documents/skin/train/"+str(d_class)+"/"+str(training[i][0])+".jpg")

NV
BCC
NV
MEL
NV
NV
BKL
BKL
BKL
BCC
BCC
NV
BKL
NV
NV
MEL
NV
MEL
NV
NV
BKL
NV
NV
NV
NV
NV
NV
NV
AKIEC
NV
NV
BKL
MEL
NV
NV
NV
NV
NV
NV
NV
VASC
NV
NV
NV
NV
NV
BKL
NV
NV
NV
NV
BCC
NV
NV
NV
NV
BKL
NV
NV
NV
NV
MEL
NV
NV
NV
MEL
NV
NV
NV
NV
NV
BCC
NV
NV
NV
AKIEC
MEL
NV
NV
NV
NV
NV
NV
DF
NV
MEL
NV
MEL
DF
NV
BKL
NV
NV
NV
NV
NV
NV
NV
NV
NV
BKL
NV
NV
NV
NV
NV
MEL
BKL
NV
NV
NV
NV
NV
NV
VASC
BKL
NV
NV
MEL
NV
NV
NV
NV
NV
NV
NV
NV
MEL
NV
BKL
NV
NV
BKL
NV
MEL
MEL
NV
MEL
NV
BCC
BKL
NV
MEL
NV
MEL
NV
NV
NV
BCC
NV
NV
NV
NV
NV
NV
NV
MEL
NV
NV
NV
NV
NV
NV
NV
NV
NV
MEL
NV
NV
MEL
NV
NV
NV
NV
VASC
MEL
MEL
BKL
NV
NV
NV
BKL
NV
MEL
BKL
NV
NV
NV
NV
BKL
NV
NV
NV
NV
NV
MEL
NV
NV
BKL
NV
AKIEC
NV
NV
NV
NV
NV
MEL
NV
NV
NV
NV
NV
NV
NV
BCC
MEL
BKL
NV
NV
NV
BKL
NV
MEL
BKL
NV
BKL
NV
NV
NV
NV
NV
NV
NV
NV
NV
AKIEC
BKL
NV
BKL
BKL
NV
NV
NV
MEL
NV
NV
NV
NV
NV
NV
NV
NV
NV
NV
NV
NV
NV
MEL
AKIEC
NV
NV
NV
NV
MEL
MEL
NV
NV
NV
NV
BCC
NV
NV
BCC
MEL
NV
NV
AKIEC
NV
NV
MEL
MEL
NV
NV
BKL
NV
MEL
NV
NV
NV
BKL
NV
NV
AKIEC
BCC
NV
NV
NV
NV
NV
N

BKL
NV
NV
BKL
MEL
BKL
BKL
NV
NV
BCC
NV
BKL
NV
NV
MEL
NV
NV
NV
NV
NV
NV
BKL
NV
NV
MEL
NV
BKL
NV
BKL
BKL
DF
NV
NV
MEL
DF
BCC
NV
MEL
NV
NV
VASC
NV
NV
NV
MEL
NV
NV
NV
NV
NV
NV
NV
DF
NV
NV
BKL
NV
NV
NV
BKL
NV
NV
BCC
NV
VASC
BCC
NV
BKL
NV
NV
NV
NV
NV
MEL
NV
NV
MEL
BKL
DF
NV
NV
NV
NV
NV
NV
NV
BCC
NV
MEL
NV
NV
NV
NV
NV
NV
NV
BCC
NV
NV
VASC
NV
NV
NV
BKL
NV
NV
MEL
NV
NV
NV
NV
NV
BCC
AKIEC
NV
NV
NV
NV
NV
NV
DF
NV
BKL
AKIEC
MEL
NV
NV
NV
BKL
BKL
NV
NV
NV
NV
NV
BKL
MEL
NV
NV
BKL
BCC
NV
MEL
NV
NV
NV
NV
BKL
NV
BKL
BKL
MEL
NV
NV
NV
VASC
NV
NV
NV
NV
NV
NV
NV
NV
NV
NV
NV
NV
BKL
BKL
MEL
NV
VASC
NV
NV
MEL
NV
BCC
NV
NV
NV
VASC
NV
BKL
NV
NV
NV
MEL
NV
NV
NV
NV
NV
MEL
NV
MEL
MEL
NV
NV
NV
MEL
BKL
NV
AKIEC
NV
NV
NV
NV
NV
NV
MEL
MEL
NV
NV
MEL
DF
NV
AKIEC
NV
NV
BKL
AKIEC
NV
BKL
NV
NV
NV
NV
NV
NV
NV
MEL
BCC
MEL
NV
NV
NV
NV
DF
NV
NV
NV
BKL
AKIEC
NV
BCC
NV
NV
NV
NV
NV
NV
NV
NV
MEL
NV
NV
BKL
NV
NV
NV
NV
BKL
NV
NV
BCC
NV
NV
DF
NV
BCC
NV
NV
NV
MEL
NV
VASC
DF
NV
MEL
BCC
MEL
NV
NV
MEL
NV
NV
NV
NV
BKL
NV
BKL
NV
NV
NV
NV
BC

BKL
NV
NV
NV
NV
BKL
NV
NV
NV
NV
NV
NV
NV
BKL
NV
NV
NV
BKL
NV
NV
NV
AKIEC
NV
NV
NV
NV
BKL
NV
BKL
NV
NV
NV
NV
NV
BCC
NV
NV
NV
NV
NV
NV
NV
BKL
BKL
NV
BCC
NV
MEL
NV
MEL
NV
MEL
NV
NV
BKL
BKL
BCC
MEL
NV
NV
NV
NV
NV
NV
NV
NV
BKL
NV
NV
NV
NV
NV
MEL
NV
BCC
NV
MEL
BKL
NV
NV
BKL
BCC
NV
NV
NV
NV
NV
NV
NV
NV
NV
NV
VASC
NV
NV
NV
NV
NV
NV
NV
NV
AKIEC
NV
BKL
MEL
NV
BCC
BCC
MEL
NV
NV
MEL
NV
MEL
NV
NV
MEL
NV
BKL
NV
NV
NV
NV
NV
NV
MEL
NV
NV
NV
NV
MEL
BCC
NV
NV
NV
VASC
NV
AKIEC
BCC
NV
NV
NV
NV
NV
NV
NV
BCC
NV
NV
NV
MEL
NV
BKL
BKL
NV
NV
MEL
NV
AKIEC
BCC
NV
NV
BKL
NV
NV
MEL
MEL
BKL
BKL
MEL
NV
VASC
BKL
BKL
NV
NV
NV
DF
AKIEC
NV
VASC
BCC
MEL
MEL
NV
BKL
NV
BKL
MEL
BKL
NV
NV
MEL
MEL
NV
NV
NV
NV
NV
NV
NV
NV
BCC
NV
NV
NV
NV
NV
DF
NV
NV
NV
NV
NV
BCC
NV
NV
NV
NV
NV
BCC
NV
NV
BCC
NV
NV
AKIEC
NV
BKL
NV
AKIEC
NV
NV
NV
NV
BKL
NV
MEL
NV
NV
NV
NV
NV
NV
NV
NV
AKIEC
BCC
BKL
MEL
MEL
NV
NV
NV
NV
NV
NV
NV
NV
AKIEC
NV
NV
NV
VASC
NV
NV
NV
NV
MEL
BCC
NV
NV
NV
BKL
NV
NV
MEL
NV
BKL
NV
AKIEC
BCC
DF
NV
DF
BKL
NV
NV
NV
MEL
NV
AKIEC
NV

In [78]:
for i in range(len(test)):
    d_class = switch_demo(test_vals[i].argmax(axis=0))
    shutil.move("/home/john/Documents/skin/data/"+test[i][0]+".jpg", "/home/john/Documents/skin/test/"+str(d_class)+"/"+str(test[i][0])+".jpg")

In [79]:
len(training)

7000

In [80]:
len(test)

3015

In [128]:
base_model = InceptionV3(weights='imagenet', include_top=False)

87924736/87910968 [==============================] - 13s 0us/step


In [130]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [133]:

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        './test',
        target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

OSError: [Errno 2] No such file or directory: './train'

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [125]:
checkpoint = ModelCheckpoint("incepV3_v1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model.fit_generator(
train_generator,
samples_per_epoch = len(training),
epochs = SHORT_EPOCH,
validation_data = validation_generator,
nb_val_samples = len(test),
callbacks = [checkpoint, early])

/home/john/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if sys.path[0] == '':
/home/john/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=437, epochs=50, callbacks=[<keras.ca..., validation_steps=3015)`
  if sys.path[0] == '':


Epoch 1/50
437/437 [==============================] - 3220s 7s/step - loss: 1.0673 - acc: 0.6638 - val_loss: 0.9296 - val_acc: 0.6684

Epoch 00001: val_acc improved from -inf to 0.66841, saving model to vgg16_1.h5
Epoch 2/50
437/437 [==============================] - 3195s 7s/step - loss: 0.8993 - acc: 0.6848 - val_loss: 0.8377 - val_acc: 0.6882

Epoch 00002: val_acc improved from 0.66841 to 0.68824, saving model to vgg16_1.h5
Epoch 3/50
437/437 [==============================] - 3191s 7s/step - loss: 0.8416 - acc: 0.6994 - val_loss: 0.8212 - val_acc: 0.6919

Epoch 00003: val_acc improved from 0.68824 to 0.69188, saving model to vgg16_1.h5
Epoch 4/50
437/437 [==============================] - 3194s 7s/step - loss: 0.7897 - acc: 0.7194 - val_loss: 0.7293 - val_acc: 0.7266

Epoch 00004: val_acc improved from 0.69188 to 0.72658, saving model to vgg16_1.h5
Epoch 5/50
437/437 [==============================] - 3202s 7s/step - loss: 0.7521 - acc: 0.7331 - val_loss: 0.7073 - val_acc: 0.7418



437/437 [==============================] - 3202s 7s/step - loss: 0.2153 - acc: 0.9185 - val_loss: 0.6098 - val_acc: 0.8043

Epoch 00042: val_acc did not improve from 0.82987
Epoch 43/50
437/437 [==============================] - 3199s 7s/step - loss: 0.1971 - acc: 0.9279 - val_loss: 0.6212 - val_acc: 0.8207

Epoch 00043: val_acc did not improve from 0.82987
Epoch 44/50
437/437 [==============================] - 3199s 7s/step - loss: 0.2031 - acc: 0.9253 - val_loss: 0.7001 - val_acc: 0.8177

Epoch 00044: val_acc did not improve from 0.82987
Epoch 45/50
437/437 [==============================] - 3199s 7s/step - loss: 0.1882 - acc: 0.9315 - val_loss: 0.5914 - val_acc: 0.8229

Epoch 00045: val_acc did not improve from 0.82987
Epoch 46/50
437/437 [==============================] - 3207s 7s/step - loss: 0.1762 - acc: 0.9348 - val_loss: 0.6483 - val_acc: 0.8192

Epoch 00046: val_acc did not improve from 0.82987
Epoch 47/50
437/437 [==============================] - 3203s 7s/step - loss: 0.155